In [1]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 10000 # Number of times the enviroment is ran
LOSS_CLIPPING = 0.2 # Approximated values stated in the original paper
LR = 0.00025 # Learning rate 
NUM_ACTIONS = 4 # Number of possible action in the environment
NUM_STATES = 24 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 32 # Batch size for the neural nets
BUFFER_SIZE = 512 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
LAMBDA = 0.9 # Used in the original paper un the GAE
NORMALIZE = True # Whether to normalize GAE or not



# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)
        
        X = Dense(512, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X_input)
        X = Dense(256, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X)
        X = Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X)
        output = Dense(NUM_ACTIONS, activation="tanh")(X)

        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss=self.ppo_loss_continuous, optimizer=OPTIMIZER(lr=LR))

    # Custom loss functions for the PPO
    def ppo_loss_continuous(self, y_true, y_pred):
        advantages, actions, logp_old_ph, = y_true[:, :1], y_true[:, 1:1+NUM_ACTIONS], y_true[:, 1+NUM_ACTIONS]
        logp = self.gaussian_likelihood(actions, y_pred)

        ratio = K.exp(logp - logp_old_ph)

        p1 = ratio * advantages
        p2 = tf.where(advantages > 0, (1.0 + LOSS_CLIPPING)*advantages, (1.0 - LOSS_CLIPPING)*advantages) # minimum advantage

        actor_loss = -K.mean(K.minimum(p1, p2))

        return actor_loss
    
    def gaussian_likelihood(self, actions, pred): # for keras custom loss
        log_std = -0.5 * np.ones(NUM_ACTIONS, dtype=np.float32)
        pre_sum = -0.5 * (((actions-pred)/(K.exp(log_std)+1e-8))**2 + 2*log_std + K.log(2*np.pi))
        return K.sum(pre_sum, axis=1)

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        V = Dense(512, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X_input)
        V = Dense(256, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(V)
        V = Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(V)
        value = Dense(1)(V)

        self.model = Model(inputs= X_input, outputs = value)
        self.model.compile(loss= 'mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
        
        self.log_std = -0.5 * np.ones(NUM_ACTIONS, dtype=np.float32)
        self.std = np.exp(self.log_std)
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_Actor_PPO_cont_v2.h5"
        self.Critic_name = f"{self.env_name}_Critic_PPO_cont_v2.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        pred = self.Actor.model.predict(state)

        low, high = -1.0, 1.0 # -1 and 1 are boundaries of tanh
        action = pred + np.random.uniform(low, high, size=pred.shape) * self.std
        action = np.clip(action, low, high)
        
        logp_t = self.gaussian_likelihood(action, pred, self.log_std)

        return action, logp_t
    
    def gaussian_likelihood(self, action, pred, log_std):
        # https://github.com/hill-a/stable-baselines/blob/master/stable_baselines/sac/policies.py
        pre_sum = -0.5 * (((action-pred)/(np.exp(log_std)+1e-8))**2 + 2*log_std + np.log(2*np.pi)) 
        return np.sum(pre_sum, axis=1)
    
    # Generalized Advantage Estimation implemented in the original paper
    def get_gaes(self, rewards, dones, values, next_values):
        # Dones are used to track when is the final step of an episode, so next values are no applied
        deltas = [r + GAMMA * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * GAMMA * LAMBDA * gaes[t + 1]

        target = gaes + values
        if NORMALIZE:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    def replay(self, states, actions, rewards, dones, next_states, logp_ts):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        logp_ts = np.vstack(logp_ts)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Get the advantage
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))

        # Stack info to unpack it in the custom loss
        y_true = np.hstack([advantages, actions, logp_ts])

        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, target, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)

 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, dones, logp_ts = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, logp_t = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action[0])
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action)
                rewards.append(reward)
                dones.append(done)
                logp_ts.append(logp_t[0])
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            
                        if average > 300:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, dones, next_states, logp_ts)
            if self.episode >= EPISODES:
                break
        self.env.close()  
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'BipedalWalker-v3'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score:  -40.51		average:  -40.51
Episode:     2		score: -115.51		average:  -78.01
Episode:     3		score: -105.95		average:  -87.33
Episode:     4		score: -114.76		average:  -94.18
Episode:     5		score: -103.88		average:  -96.12
Episode:     6		score: -109.78		average:  -98.40
Episode:     7		score:  -44.03		average:  -90.63
Episode:     8		score: -102.46		average:  -92.11
Episode:     9		score:  -48.42		average:  -87.26
Episode:    10		score:  -45.48		average:  -83.08
Episode:    11		score: -113.22		average:  -85.82
Episode:    12		score:  -99.78		average:  -86.98
Episode:    13		score:  -48.57		average:  -84.03
Episode:    14		score:  -42.44		average:  -81.06
Episode:    15		score:  -98.95		average:  -82.25
Episode:    16		score:  -43.27		average:  -79.81
Episode:    17		score:  -49.81		average:  -78.05
Episode:    18		score:  -94.98		average:  -78.99
Episode:    19		score: -117.36		avera

Episode:   162		score:  -43.62		average:  -71.74
Episode:   163		score:  -50.55		average:  -71.78
Episode:   164		score:  -39.09		average:  -71.73
Episode:   165		score:  -96.97		average:  -71.56
Episode:   166		score:  -40.48		average:  -70.87
Episode:   167		score:  -39.92		average:  -70.26
Episode:   168		score:  -38.13		average:  -70.20
Episode:   169		score:  -47.59		average:  -70.23
Episode:   170		score:  -35.35		average:  -70.23
Episode:   171		score:  -42.51		average:  -70.23
Episode:   172		score: -122.35		average:  -71.05
Episode:   173		score:  -38.39		average:  -71.06
Episode:   174		score: -112.54		average:  -71.76
Episode:   175		score: -112.52		average:  -72.51
Episode:   176		score:  -36.89		average:  -72.52
Episode:   177		score:  -41.91		average:  -72.58
Episode:   178		score:  -38.59		average:  -72.53
Episode:   179		score:  -99.01		average:  -72.49
Episode:   180		score: -110.98		average:  -73.22
Episode:   181		score:  -40.09		average:  -72.66
Episode:   182		scor

Episode:   330		score: -101.62		average:  -65.69
Episode:   331		score: -115.67		average:  -66.42
Episode:   332		score: -111.46		average:  -66.55
Episode:   333		score:  -39.80		average:  -65.76
Episode:   334		score:  -38.80		average:  -65.81
Episode:   335		score:  -36.62		average:  -65.02
Episode:   336		score: -109.85		average:  -65.11
Episode:   337		score:  -36.43		average:  -65.10
Episode:   338		score: -114.42		average:  -65.13
Episode:   339		score: -102.96		average:  -65.80
Episode:   340		score:  -37.46		average:  -65.73
Episode:   341		score:  -36.71		average:  -65.65
Episode:   342		score:  -38.66		average:  -65.60
Episode:   343		score:  -36.98		average:  -65.59
Episode:   344		score:  -42.25		average:  -64.96
Episode:   345		score:  -35.85		average:  -64.85
Episode:   346		score:  -42.73		average:  -64.31
Episode:   347		score: -118.05		average:  -65.06
Episode:   348		score:  -46.01		average:  -64.53
Episode:   349		score:  -31.66		average:  -64.40
Episode:   350		scor

Episode:   498		score:  -31.62		average:  -46.16
Episode:   499		score:  -29.63		average:  -45.50
Episode:   500		score:  -29.42		average:  -45.36
Episode:   501		score:  -36.37		average:  -45.38
Episode:   502		score:  -29.00		average:  -45.34
Episode:   503		score:  -36.58		average:  -44.51
Episode:   504		score:  -27.17		average:  -43.82
Episode:   505		score:  -34.78		average:  -43.87
Episode:   506		score:  -31.86		average:  -43.22
Episode:   507		score:  -30.40		average:  -43.15
Episode:   508		score:  -25.11		average:  -43.05
Episode:   509		score:  -33.54		average:  -43.09
Episode:   510		score:  -33.17		average:  -43.06
Episode:   511		score:  -30.63		average:  -43.05
Episode:   512		score:  -33.95		average:  -43.06
Episode:   513		score:  -27.76		average:  -42.95
Episode:   514		score:  -33.51		average:  -42.91
Episode:   515		score:  -30.53		average:  -42.87
Episode:   516		score:  -25.09		average:  -42.79
Episode:   517		score:  -34.35		average:  -42.73
Episode:   518		scor

Episode:   666		score:  -30.54		average:  -44.16
Episode:   667		score:  -20.53		average:  -44.14
Episode:   668		score:  -27.60		average:  -43.24
Episode:   669		score:  -26.68		average:  -42.43
Episode:   670		score:  -21.92		average:  -41.48
Episode:   671		score:  -26.09		average:  -41.53
Episode:   672		score:  -25.49		average:  -41.60
Episode:   673		score: -104.22		average:  -41.70
Episode:   674		score:  -22.67		average:  -40.77
Episode:   675		score:  -30.10		average:  -40.16
Episode:   676		score:  -34.06		average:  -40.20
Episode:   677		score:  -28.23		average:  -40.18
Episode:   678		score: -104.43		average:  -40.06
Episode:   679		score:  -36.97		average:  -40.17
Episode:   680		score: -100.74		average:  -40.87
Episode:   681		score:  -31.54		average:  -40.86
Episode:   682		score: -100.09		average:  -41.60
Episode:   683		score:  -97.29		average:  -41.45
Episode:   684		score:  -32.08		average:  -41.49
Episode:   685		score:  -33.91		average:  -41.54
Episode:   686		scor

...

Episode:  9238		score:  281.18		average:  219.98
Episode:  9239		score:  277.87		average:  221.00
Episode:  9240		score:  280.88		average:  220.98
Episode:  9241		score:  276.13		average:  220.95
Episode:  9242		score:  270.31		average:  220.83
Episode:  9243		score:  281.42		average:  220.85
Episode:  9244		score:  281.34		average:  220.83
Episode:  9245		score:  287.86		average:  220.85
Episode:  9246		score:  281.78		average:  220.78
Episode:  9247		score:  267.12		average:  220.66
Episode:  9248		score:  275.34		average:  220.57
Episode:  9249		score:  270.19		average:  224.25
Episode:  9250		score:  277.36		average:  224.18
Episode:  9251		score:  278.00		average:  224.13
Episode:  9252		score:  273.78		average:  227.32
Episode:  9253		score:  158.65		average:  226.16
Episode:  9254		score:   38.35		average:  223.76
Episode:  9255		score:  272.96		average:  227.47
Episode:  9256		score:  278.22		average:  227.41
Episode:  9257		score:  279.03		average:  227.37
Episode:  9258		scor

Episode:  9406		score:  281.72		average:  187.81
Episode:  9407		score:  288.04		average:  187.81
Episode:  9408		score:  275.98		average:  187.71
Episode:  9409		score:  277.97		average:  187.68
Episode:  9410		score:  284.36		average:  191.45
Episode:  9411		score:  287.85		average:  194.09
Episode:  9412		score:  286.17		average:  194.16
Episode:  9413		score:  281.58		average:  194.29
Episode:  9414		score:  -85.42		average:  190.62
Episode:  9415		score:  289.28		average:  190.76
Episode:  9416		score:  284.19		average:  190.82
Episode:  9417		score:  283.82		average:  190.84
Episode:  9418		score:  269.60		average:  190.77
Episode:  9419		score:   95.16		average:  192.51
Episode:  9420		score:  272.66		average:  192.37
Episode:  9421		score: -106.68		average:  188.38
Episode:  9422		score:  -36.87		average:  185.15
Episode:  9423		score:  278.64		average:  185.09
Episode:  9424		score:  277.29		average:  185.01
Episode:  9425		score:  283.64		average:  185.02
Episode:  9426		scor

Episode:  9574		score:  285.87		average:  138.72
Episode:  9575		score:  282.44		average:  142.53
Episode:  9576		score:  -96.85		average:  140.01
Episode:  9577		score:  -30.25		average:  140.80
Episode:  9578		score:  -97.72		average:  136.92
Episode:  9579		score:   88.11		average:  137.72
Episode:  9580		score:   88.89		average:  139.72
Episode:  9581		score:  281.81		average:  139.70
Episode:  9582		score:  282.25		average:  143.55
Episode:  9583		score:  269.96		average:  144.53
Episode:  9584		score:  279.30		average:  148.47
Episode:  9585		score:  274.21		average:  152.24
Episode:  9586		score:  278.69		average:  156.12
Episode:  9587		score:  280.01		average:  156.10
Episode:  9588		score:  276.44		average:  159.27
Episode:  9589		score:  280.48		average:  163.17
Episode:  9590		score: -101.28		average:  159.28
Episode:  9591		score:  278.29		average:  159.20
Episode:  9592		score:  272.84		average:  159.12
Episode:  9593		score:  277.56		average:  159.03
Episode:  9594		scor

Episode:  9742		score:  290.26		average:  165.00
Episode:  9743		score:  -87.53		average:  161.25
Episode:  9744		score:  287.80		average:  161.24
Episode:  9745		score:  288.80		average:  161.26
Episode:  9746		score:    7.35		average:  158.42
Episode:  9747		score:  290.54		average:  158.42
Episode:  9748		score:  281.82		average:  158.32
Episode:  9749		score:  284.44		average:  158.30
Episode:  9750		score:  296.13		average:  158.37
Episode:  9751		score: -110.99		average:  154.38
Episode:  9752		score:  290.72		average:  157.22
Episode:  9753		score:  287.23		average:  157.20
Episode:  9754		score:    2.43		average:  154.30
Episode:  9755		score: -105.83		average:  150.37
Episode:  9756		score:  128.19		average:  148.71
Episode:  9757		score:  290.18		average:  149.82
Episode:  9758		score:  -87.00		average:  146.00
Episode:  9759		score:  289.92		average:  146.02
Episode:  9760		score:  283.74		average:  145.98
Episode:  9761		score:  285.86		average:  145.95
Episode:  9762		scor

Episode:  9910		score:  -30.11		average:  171.35
Episode:  9911		score:  -89.25		average:  167.59
Episode:  9912		score:   63.46		average:  165.37
Episode:  9913		score:  276.80		average:  165.22
Episode:  9914		score:  280.91		average:  165.17
Episode:  9915		score:  273.92		average:  165.01
Episode:  9916		score:  281.47		average:  164.95
Episode:  9917		score:  284.34		average:  164.92
Episode:  9918		score:  -44.75		average:  161.60
Episode:  9919		score:  281.68		average:  161.61
Episode:  9920		score:  278.45		average:  161.54
Episode:  9921		score:  -11.37		average:  162.09
Episode:  9922		score:  282.77		average:  162.04
Episode:  9923		score:  285.68		average:  162.02
Episode:  9924		score: -101.78		average:  158.15
Episode:  9925		score:   48.81		average:  155.79
Episode:  9926		score:  283.53		average:  158.96
Episode:  9927		score:  283.48		average:  158.92
Episode:  9928		score: -103.45		average:  155.03
Episode:  9929		score: -103.83		average:  152.12
Episode:  9930		scor